# Erdös Institute
## Data Science Boot Camp Project: Police Interaction
--- 

In [1]:
import pandas as pd
df = pd.read_csv('/Users/patrickma1994/Documents/data.tsv', sep='\t',dtype='str')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130122 entries, 0 to 130121
Columns: 277 entries, SCRMCTRL to V353
dtypes: object(277)
memory usage: 275.0+ MB


In [2]:
# Checking if the value satisfy the codebook
print(df['V347'].value_counts())
list(df.columns)

V347
9     112008
1      16628
2        903
3        529
8         40
98        14
Name: count, dtype: int64


['SCRMCTRL',
 'PPCSWGT',
 'SEQNUM',
 'SEX',
 'AGE',
 'INTTYPE',
 'NONINT',
 'HISP',
 'MODE',
 'PSSTRATA',
 'SECUCODE',
 'INCOME',
 'V10A',
 'V10B',
 'V10C',
 'V10D',
 'V10E',
 'V10F',
 'V10G',
 'V10H',
 'V10I',
 'V10J',
 'V10K',
 'V10L',
 'V10M',
 'V10N',
 'V22A',
 'V22B',
 'V22C',
 'V22D',
 'V22E',
 'V22F',
 'V22G',
 'V22H',
 'V22I',
 'V22J',
 'V22K',
 'V22L',
 'V22M',
 'V22N',
 'V22O',
 'V22P',
 'V23A',
 'V23B',
 'V23C',
 'V23D',
 'V23E',
 'V23F',
 'V23G',
 'V23H',
 'V23I',
 'V23J',
 'V23K',
 'V23L',
 'V23M',
 'V23N',
 'V23O',
 'V23P',
 'V23Q',
 'V23R',
 'V23S',
 'V1',
 'V23T',
 'V23U',
 'V23V',
 'V23W',
 'V23X',
 'V23Y',
 'V23Z',
 'V24A',
 'V24B',
 'V24C',
 'V24J',
 'V24D',
 'V24E',
 'V24F',
 'V24G',
 'V24H',
 'V24I',
 'CHECK_ITEM_B',
 'CHECK_ITEM_B1A',
 'CHECK_ITEM_B1B',
 'CHECK_ITEM_B1C',
 'CHECK_ITEM_B1D',
 'CHECK_ITEM_B1E',
 'CHECK_ITEM_B1F',
 'CHECK_ITEM_B1G',
 'CHECK_ITEM_B1H',
 'CHECK_ITEM_B1I',
 'CHECK_ITEM_B1J',
 'CHECK_ITEM_B1K',
 'V32',
 'CHECK_ITEM_B2A',
 'CHECK_ITEM_B2B

In [3]:
df_m = df[['AGE', 'HISP', 'SEX', 'INCOME', 'V347']].rename(columns={'V347': 'proper_behave'})
df_m.head()

,AGE,HISP,SEX,INCOME,proper_behave
0,3,3,2,3,9
1,3,3,1,3,9
2,3,1,2,4,9
3,3,1,1,4,9
4,4,1,2,3,9


In [4]:
# Convert columns to categorical
df_m['AGE'] = pd.Categorical(df_m['AGE'])
df_m['HISP'] = pd.Categorical(df_m['HISP'])
df_m['SEX'] = pd.Categorical(df_m['SEX'])
df_m['INCOME'] = pd.Categorical(df_m['INCOME'])

# Filter rows where proper_behave is 1 or 2
df_m = df_m[df_m['proper_behave'].isin(['1', '2'])]

# Remap proper_behave values
df_m['proper_behave'] = df_m['proper_behave'].map({'1': 1, '2': 0})

# Convert proper_behave to numeric and checking the column number
#df_m['proper_behave'] = pd.to_numeric(df_m['proper_behave'])
df_m.shape 
#df_m.head()
# Check missing values in all columns
print("Missing values in each column:")
print(df_m.isnull().sum())
# Seems to be no missing values to the current dataset

Missing values in each column:
AGE              0
HISP             0
SEX              0
INCOME           0
proper_behave    0
dtype: int64


The following codes drops missing values, but our data doesn't have missing values for the features selected. 

In [6]:
# Drop rows with missing values in AGE, HISP, SEX, and INCOME
df_m = df_m.dropna(subset=['AGE', 'HISP', 'SEX', 'INCOME'])

# Verify the shape after dropping missing values
print("\nDataFrame shape after dropping missing values:")
print(df_m.shape)

# Verify there are no more missing values in these columns
print("\nRemaining missing values:")
print(df_m[['AGE', 'HISP', 'SEX', 'INCOME']].isnull().sum())


DataFrame shape after dropping missing values:
(17531, 5)

Remaining missing values:
AGE       0
HISP      0
SEX       0
INCOME    0
dtype: int64
